In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!pip install argparse
!pip install re
!pip install emoji
!pip install pickle
!pip install gensim

  Could not find a version that satisfies the requirement re (from versions: )
No matching distribution found for re
  Could not find a version that satisfies the requirement pickle (from versions: )
No matching distribution found for pickle


In [0]:
#for parsing
import argparse

#for regex
import re

#The entire set of Emoji codes as defined 
import emoji

import numpy as np
import pandas as pd

#erializing and de-serializing python object structure
import pickle

#topic modelling, document indexing and similarity retrieval with large corpora
#Target audience is being natural language processing
from gensim.parsing.preprocessing import *

In [0]:
#Created Models being Imported

#word2vec
#mean_title_embedding
#mean_log_video_views
#mean_log_video_likes
#mean_log_video_dislikes
#mean_log_video_comments
#min_max_scaler

#Prediction Model Used to predict the Result---------
#svm

In [0]:

#method for tokenization a string in python

def tokenize(string):

    """ Tokenizes a string.

    Returns a list of stems and, eventually, emojis.
    """

    # Based on the Ranks NL (Google) stopwords list, but "how" and "will" are not stripped, and
    # words shorter than 2 characters are not checked (since they are stripped):
    
    stop_words = [
        "about", "an", "are", "as", "at", "be", "by", "com", "for", "from", "in", "is", "it", "of",
        "on", "or", "that", "the", "this", "to", "was", "what", "when", "where", "who", "with",
        "the", "www"
    ]

    
    #Stripping the  whitespaces and alphanumerics
    string = strip_short(
        strip_multiple_whitespaces(
            strip_punctuation(
                split_alphanum(string))),
        minsize=2)
    
    # Parse emojis:
    emojis = [c for c in string if c in emoji.UNICODE_EMOJI]
    
    # Remove every non-word character and stem each word:
    string = stem_text(re.sub(r"[^\w\s,]", "", string))
    
    # List of stems and emojis:
    tokens = string.split() + emojis

    for stop_word in stop_words:
        try:
            tokens.remove(stop_word)
        except:
            pass

    return tokens


def average_embedding(tokens, word2vec, na_vector=None):

    """ Embeds a title with the average representation of its tokens.

    Returns the mean vector representation of the tokens representations. When no token is in the
    """

    vectors = list()

    for token in tokens:
        if token in word2vec:
            vectors.append(word2vec[token])

    if len(vectors) == 0 and na_vector is not None:
        vectors.append(na_vector)

    return np.mean(np.array(vectors), axis=0)


# Import the Word2Vec model and the mean vector representation computed on the train set:
word2vec = pickle.load(open("/content/drive/My Drive/youtube/data/word2vec", "rb"))
mean_title_embedding = pickle.load(open("/content/drive/My Drive/youtube/data/mean-title-embedding", "rb"))

#User input here:
input = {
    "video_title": "New York",
    "video_views": 200000000,
    "video_likes": 200,
    "video_dislikes": 200,
    "video_comments": 10,
}
youtube_samples = pd.DataFrame([ input ])

# Tokenize the title and then compute its embedding:
youtube_samples["video_title"] = youtube_samples["video_title"].apply(tokenize)
youtube_samples["video_title"] = youtube_samples["video_title"].apply(
    average_embedding, word2vec=word2vec, na_vector=mean_title_embedding)

#concatenate the views, likes, dislikes, comments
youtube_samples = pd.concat(
    [
        youtube_samples[["video_views", "video_likes", "video_dislikes", "video_comments"]],
        youtube_samples["video_title"].apply(pd.Series)
    ], axis=1)

# Compute the log of the video metadata or replace the missing values with the mean values obtained
# from the train set:
mean_log_video_views = pickle.load(open("/content/drive/My Drive/youtube/data/mean-log-video-views", "rb"))
mean_log_video_likes = pickle.load(open("/content/drive/My Drive/youtube/data/mean-log-video-likes", "rb"))
mean_log_video_dislikes = pickle.load(open("/content/drive/My Drive/youtube/data/mean-log-video-dislikes", "rb"))
mean_log_video_comments = pickle.load(open("/content/drive/My Drive/youtube/data/mean-log-video-comments", "rb"))

#Data Preprocessing here

#Data Transformation
youtube_samples[["video_views", "video_likes", "video_dislikes", "video_comments"]] = \
    youtube_samples[["video_views", "video_likes", "video_dislikes", "video_comments"]].apply(np.log)

#If any null itemset then check for them 
if youtube_samples["video_views"].isnull().any():
    youtube_samples["video_views"].fillna(mean_log_video_views, inplace=True)
    
    
if youtube_samples["video_likes"].isnull().any():
    youtube_samples["video_likes"].fillna(mean_log_video_likes, inplace=True)
    
if youtube_samples["video_dislikes"].isnull().any():
    youtube_samples["video_dislikes"].fillna(mean_log_video_dislikes, inplace=True)
    
if youtube_samples["video_comments"].isnull().any():
    youtube_samples["video_comments"].fillna(mean_log_video_comments, inplace=True)

# Replace any -Inf value with 0:
youtube_samples = youtube_samples.replace(-np.inf, 0)

# Import the min-max scaler and apply it to the youtube_samples:
min_max_scaler = pickle.load(open("/content/drive/My Drive/youtube/data/min-max-scaler", "rb"))
youtube_samples = pd.DataFrame(min_max_scaler.transform(youtube_samples), columns=youtube_samples.columns)

# Import the SVM model:
svm = pickle.load(open("/content/drive/My Drive/youtube/data/svm", "rb"))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.19.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVC from version 0.19.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [0]:
#Click Bait(1) or Not(0)


# Print its prediction:
print(svm.predict(youtube_samples)[0])

0
